# Dataset preprocessing

The goal of this notebook is to create a preprocessed kaggle dataset out of the competition dataset.  
For now, the preprocessing will be based on [this notebook](https://www.kaggle.com/code/vonmainstein/imu-tof).  
It consists of the following steps:
-   Set the appropriate dtypes.
-   forward + backward filling feature columns
-   Converting the files csv files to parquet for faster reload and getting the correct data types upon loading.
-   Output statistics of the dataset into a json file to enable the model to normalize the input at run time.  

> Note:  
> - The padding of the sequences will be performed in the model since we don't have access to the "leaderboard dataset" inputs.  
> - Demographics data set will be ignored for now.  

## Imports

In [15]:
from os.path import join

import pandas as pd
from pandas import DataFrame as DF
from kagglehub import whoami, competition_download, dataset_upload

from config import COMPETITION_HANDLE, DATASET_DF_DTYPES, META_DATA_COLUMNS

## Obtain raw dataset
Requires to be logged in if this notebook is not running on laggle, go to [your settings](https://www.kaggle.com/settings) to create an access token and put it in `~/.kaggle/`.

In [2]:
competition_dataset_path = competition_download(COMPETITION_HANDLE)

In [4]:
df = pd.read_csv(join(competition_dataset_path, "train.csv"), dtype=DATASET_DF_DTYPES)

In [ ]:
features_describe = (
    df
    .drop(META_DATA_COLUMNS, axis="columns")
    .describe()
)
features_describe

,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,rot_z,thm_1,thm_2,thm_3,...,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
count,574945.000000,574945.000000,574945.000000,571253.000000,571253.000000,571253.000000,571253.000000,567958.000000,567307.000000,568473.000000,...,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000
mean,1.639980,1.790704,-0.459811,0.360375,-0.119916,-0.059953,-0.188298,27.076450,27.133483,26.702994,...,29.395651,26.030826,45.342583,43.074842,40.045908,37.631707,34.977928,31.934330,29.024752,27.320358
std,5.781259,5.003945,6.096490,0.225739,0.465520,0.543028,0.504137,3.231948,2.941438,4.122353,...,58.093844,54.215523,68.466064,68.017631,66.941587,65.288710,63.201604,60.440645,57.218513,55.407192
min,-34.585938,-24.402344,-42.855469,0.000000,-0.999146,-0.999695,-0.998169,-0.370413,21.958820,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-2.964844,-2.121094,-5.417969,0.180237,-0.456299,-0.511536,-0.627686,24.753527,24.543737,24.640350,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,2.972656,0.695312,-1.562500,0.340332,-0.186890,-0.112610,-0.263916,26.982324,26.354338,26.956276,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
75%,6.343750,6.816406,5.164062,0.503479,0.204590,0.440063,0.251099,29.425036,29.620147,29.231794,...,34.000000,24.000000,81.000000,76.000000,67.000000,59.000000,51.000000,42.000000,35.000000,31.000000
max,46.328125,27.183594,30.078125,0.999390,0.999817,0.999451,0.999878,38.457664,37.578339,37.294994,...,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000


In [19]:
! mkdir -p preprocessed_dataset

df.to_parquet("preprocessed_dataset/train.parquet", index=False)
features_describe.to_csv("preprocessed_dataset/features_describe.csv")

In [20]:
df.shape

(574945, 341)

In [ ]:
dataset_upload(
    join(whoami()["username"], "prepocessed-CMI-2025"),
    "preprocessed_dataset",
    version_notes="Preprocessed Child Mind "
)

Kaggle credentials successfully validated.
Uploading Dataset https://www.kaggle.com/datasets/mauroabidalcarrer/prepocessed-CMI-2025 ...
Starting upload for file preprocessed_dataset/train.parquet


Uploading: 100%|██████████| 127M/127M [01:10<00:00, 1.81MB/s] 

Upload successful: preprocessed_dataset/train.parquet (121MB)
Starting upload for file preprocessed_dataset/features_describe.csv



Uploading: 100%|██████████| 27.6k/27.6k [00:00<00:00, 68.1kB/s]

Upload successful: preprocessed_dataset/features_describe.csv (27KB)


Your dataset has been created.
Files are being processed...
See at: https://www.kaggle.com/datasets/mauroabidalcarrer/prepocessed-CMI-2025
